# This is the notebook we use to test our queries. For some queries, we need to get condition from frontend. So we make them as  functions. For others, we will just show the results. 

In [1]:
# pymysql is the library in python to connect mysql. 
#In this notebook, I have already provide you with the username, password, host and database

import pymysql as pm
name  = 'cywdatabase.ccszxqpi5wa6.us-east-1.rds.amazonaws.com'
db = pm.connect(host = name,user = 'chenyunw',password = 'z1x2c3v4',db = 'cis550')
cursor = db.cursor()

schema for post/reg players:
['PLAYER_ID','PLAYER','TEAM','GP','MIN','FG_PCT','FG3_PCT','FT_PCT','REB','AST','STL','BLK','TOV','EFF','PTS','year']
where PLAYER_ID,TEAM,year are keys
schema for post/reg 
['TEAM_ID','TEAM_NAME','W','L','W_PCT','FG_PCT','FG3_PCT','FT_PCT','REB','AST','STL','BLK','TOV','PTS','year']
where TEAM_ID,TEAM_NAME,year are keys

schema for ratings:
Team,Player,Position,Overall ,Height,Weight,Tier,Program,Playstyle,Speed,Midrange,Finishing ,3Pt,Dribble,PostScoringt,Rebound ,Defense
schema for salary:
'Player','2018season','2019season','2020season','2021season'


The 6 tables are :
(('PostSeasonPlayer',),
 ('PostSeasonTeam',),
 ('Ratings',),
 ('RegSeasonPlayer',),
 ('RegSeasonTeam',),
 ('Salary',))

## Basic Queries:
in the following cells, we will write some basic queries. For example, we will present information of a certain player or team. W e will also try to add index to see whether it can be searched faster. 

In [2]:
#This is the Basic Query for player. In practise, we will add more fields such as STL and BLK and FG_PCT.  
#But here, we just show how we do if some fields are missing.

def Player(name,PTS = None,AST = None, REB = None,year = None,Reg = True):
    pts = ''
    ast = ''
    reb = ''
    y = ''
    season = 'PostSeasonPlayer'
    if PTS != None:
        pts = ' and PTS = ' + str(PTS)
    if AST != None:
        ast = ' and AST = ' + str(AST)
    if REB !=None:
        reb = ' and REB = ' +str(AST)
    if year != None:
        y = ' and year = ' + str(year)
    if Reg:
        season = 'RegSeasonPlayer'
    name = "PLAYER = " +"'" +name+"'"
    
    query = 'SELECT PLAYER,PTS,AST,REB,year,TEAM FROM ' + season + ' WHERE ' +name + pts+ast+reb+y
#     print(query)
    cursor.execute(query)
    return cursor.fetchall()

In [3]:
#Basic Query for player. We can revise the return field 
%time

Player('James Harden',Reg = True)
# CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
# Wall time: 6.91 µs
    
    
    

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


(('James Harden', 25.9, 5.8, 4.9, 2012, 'Houston Rockets'),
 ('James Harden', 25.4, 6.1, 4.7, 2013, 'Houston Rockets'),
 ('James Harden', 27.4, 7.0, 5.7, 2014, 'Houston Rockets'),
 ('James Harden', 29.0, 7.5, 6.1, 2015, 'Houston Rockets'),
 ('James Harden', 29.1, 11.2, 8.1, 2016, 'Houston Rockets'),
 ('James Harden', 30.4, 8.8, 5.4, 2017, 'Houston Rockets'),
 ('James Harden', 9.9, 1.8, 3.2, 2009, 'Oklahoma City Thunder'),
 ('James Harden', 12.2, 2.1, 3.1, 2010, 'Oklahoma City Thunder'),
 ('James Harden', 16.8, 3.7, 4.1, 2011, 'Oklahoma City Thunder'))

In [4]:
#Basic Query for team
def Player(TEAM,PTS = None,AST = None, REB = None,year = None,Reg = True):
    pts = ''
    ast = ''
    reb = ''
    y = ''
    season = 'PostSeasonTeam'
    if PTS != None:
        pts = ' and PTS = ' + str(PTS)
    if AST != None:
        ast = ' and AST = ' + str(AST)
    if REB !=None:
        reb = ' and REB = ' +str(AST)
    if year != None:
        y = ' and year = ' + str(year)
    if Reg:
        season = 'RegSeasonTeam'
    name = "TEAM_NAME = " +"'" +TEAM+"'"

    query = 'SELECT TEAM_NAME,PTS,AST,REB,year FROM ' + season + ' WHERE ' +name + pts+ast+reb+y
    print(query)
    cursor.execute(query)
    result = cursor.fetchall()
    return result

#example
Player('Houston Rockets',Reg = False)

SELECT TEAM_NAME,PTS,AST,REB,year FROM PostSeasonTeam WHERE TEAM_NAME = 'Houston Rockets'


(('Houston Rockets', 84.6, 17.2, 44.4, 2003),
 ('Houston Rockets', 97.6, 18.0, 37.7, 2004),
 ('Houston Rockets', 87.3, 18.1, 41.6, 2006),
 ('Houston Rockets', 88.0, 15.5, 42.0, 2007),
 ('Houston Rockets', 91.5, 17.0, 40.5, 2008),
 ('Houston Rockets', 100.0, 18.0, 43.5, 2012),
 ('Houston Rockets', 112.0, 18.3, 49.2, 2013),
 ('Houston Rockets', 108.2, 21.9, 45.7, 2014),
 ('Houston Rockets', 91.2, 18.0, 47.4, 2015),
 ('Houston Rockets', 107.7, 19.5, 42.9, 2016),
 ('Houston Rockets', 104.9, 18.8, 43.0, 2017))

In [7]:
# total average attributes through 21st century
team = 'Houston Rockets'
sql = "select TEAM_NAME,format(AVG(W),2) as avrage_win,format(AVG(L),2) as average_loss,format(SUM(W)/(SUM(W+L)),2) as average_WPCT \
,format(SUM((W+L)*FG_PCT)/SUM(W+L),2) AS average_FGPCT \
,format(SUM((W+L)*PTS)/SUM(W+L),2) as average_pts,format(SUM((W+L)*AST)/SUM(W+L),2) as average_ast \
,format(SUM((W+L)*REB)/SUM(W+L),2) as average_REB FROM RegSeasonTeam Where TEAM_NAME = '"+team+"' GROUP BY TEAM_NAME"
cursor.execute(sql)
cursor.fetchall()

(('Houston Rockets',
  '46.72',
  '34.39',
  '0.58',
  '0.45',
  '100.50',
  '21.17',
  '43.10'),)

In [5]:
#average fields of a player
#in the player person page, we will show the average data. Though some information are missing , such as PCT, we will use the same fucntion to
#calculate that
def average_pts(name):
    name_search = 'PLAYER = ' + "'"+name +"'"
    sql = 'SELECT PLAYER, SUM(GP*PTS)/SUM(GP) as average_pts, \
    SUM(GP*AST)/SUM(GP) as average_ast,SUM(GP*REB)/SUM(GP) as average_reb, \
    SUM(GP*STL)/SUM(GP) as average_stl, SUM(GP*BLK)/SUM(GP) as average_blk, \
    SUM(GP*FG_PCT)/SUM(GP) as average_pct,SUM(GP*FG3_PCT)/SUM(GP) as average_3pct \
    FROM RegSeasonPlayer WHERE '+name_search + 'GROUP BY PLAYER'
    cursor.execute(sql)
    return cursor.fetchall()

name = 'Kobe Bryant'
average_pts(name)

(('Kobe Bryant',
  27.46275264053418,
  5.083445621064584,
  5.549951897548643,
  1.5214629458968059,
  0.4367661285245499,
  0.44815880685499704,
  0.32778248411744),)

## Do we need index in this database? It seems since we will sometimes query on PTS, but let's look at how it performs with/without index

In [10]:
import timeit
sql = 'select PLAYER ,PTS FROM RegSeasonPlayer where PTS >22.5 and PTS <30 LIMIT 5'
start_time = timeit.default_timer()
for i in range(1000):
    cursor.execute(sql)
elapsed = timeit.default_timer() - start_time
elapsed
#while running for 1000times,we use 8.522s

8.522034502006136

In [13]:
index ='CREATE INDEX multi_index on RegSeasonPlayer(PTS,AST,REB)'
cursor.execute(index)

0

In [14]:
import timeit
sql = 'select PLAYER ,PTS FROM RegSeasonPlayer where PTS >22.5 and PTS <30 LIMIT 5'
start_time = timeit.default_timer()
for i in range(1000):
    cursor.execute(sql)
elapsed = timeit.default_timer() - start_time
elapsed

#From the result , we find after adding index, running 1000 times improves by only 0.04s,which means it improves 0.00004 for each query
#we know index will improve our speed, but it seems it wont improve much for a small database.
#I also add sql to drop them, I keep them now but you can delete if you want.

8.482392670004629

In [12]:
# drop_index = 'DROP INDEX multi_index on RegSeasonPlayer'
# cursor.execute(drop_index)

0

# Advanced Queries

In [15]:
#Query 1:
#Who is the best postseason type player? 
#Which means he has a better performance in post season than in regular season by points, assists, and Reb.

def better_player(compare):
    sql1 = None
    if compare == 'PTS':
        sql1 = 'select rp.player,(pp.PTS/rp.PTS) as PTS_RATIO, rp.year,pp.PTS,rp.PTS \
        FROM RegSeasonPlayer rp join PostSeasonPlayer pp on rp.PLAYER_ID = pp.PLAYER_ID and pp.year = rp.year \
        WHERE rp.PTS > 20 \
        ORDER BY PTS_RATIO DESC LIMIT 10'
    elif compare == 'AST':
        sql1 = 'select rp.player,(pp.AST/rp.AST) as AST_RATIO, rp.year,pp.AST,rp.AST \
        FROM RegSeasonPlayer rp join PostSeasonPlayer pp on rp.PLAYER_ID = pp.PLAYER_ID and pp.year = rp.year \
        WHERE rp.AST > 8 \
        ORDER BY AST_RATIO DESC LIMIT 10'
    elif compare == 'REB':
        sql1 = 'select rp.player,(pp.REB/rp.REB) as REB_RATIO, rp.year,pp.REB,rp.REB \
        FROM RegSeasonPlayer rp join PostSeasonPlayer pp on rp.PLAYER_ID = pp.PLAYER_ID and pp.year = rp.year \
        WHERE rp.REB > 8 \
        ORDER BY REB_RATIO DESC LIMIT 10'
    return sql1

compare = 'AST'
sql1 = better_player(compare)
sql1
if sql1 != None:
    cursor.execute(sql1)
    result = cursor.fetchall()
result

(('Rajon Rondo', 1.4878048893952927, 2017, 12.2, 8.2),
 ('Jason Kidd', 1.3571428501918783, 2000, 13.3, 9.8),
 ('John Stockton', 1.3103448124665187, 2000, 11.4, 8.7),
 ('John Stockton', 1.2195122234882314, 2001, 10.0, 8.2),
 ('Rajon Rondo', 1.1951220022788165, 2008, 9.8, 8.2),
 ('Jason Kidd', 1.1847825917945238, 2006, 10.9, 9.2),
 ('Chris Paul', 1.1734693649161716, 2010, 11.5, 9.8),
 ('Steve Nash', 1.1465517028758705, 2006, 13.3, 11.6),
 ('Jason Kidd', 1.142857240170853, 2005, 9.6, 8.4),
 ('Chris Paul', 1.076086937367127, 2016, 9.9, 9.2))

In [2]:
#Query2

# as  PTS will appear in most where query,we set PTS as index
# sql_index_PTS = 'create index index_PTS on RegSeasonPlayer(PTS)'
# # as 2018season salary may appear in most where querys, we set it as index
# sql_index_salary = 'create index index_salary on Salary(2018season)'
# # after set index,we just comment these sql
# cursor.execute(sql_index_PTS)
# cursor.execute(sql_index_salary)
cursor.execute('show keys from RegSeasonPlayer')
cursor.fetchall()

(('RegSeasonPlayer',
  0,
  'PRIMARY',
  1,
  'PLAYER_ID',
  'A',
  920,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  0,
  'PRIMARY',
  2,
  'TEAM',
  'A',
  1903,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  0,
  'PRIMARY',
  3,
  'year',
  'A',
  3707,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  1,
  'TEAM',
  1,
  'TEAM',
  'A',
  36,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  1,
  'TEAM',
  2,
  'year',
  'A',
  536,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  1,
  'multi_index',
  1,
  'PTS',
  'A',
  289,
  None,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  1,
  'multi_index',
  2,
  'AST',
  'A',
  2818,
  None,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES'),
 ('RegSeasonPlayer',
  1,
  'multi_index',
  3,
  'REB',
  'A',
  3683,
  None,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES

In [17]:
# Query2
# Among those top players, whose contract has best price–performance ratio? Whose has the worst?
# To make it simple, we assume he must get at least 20 points, and his salary/pts is very high.

sql2 = 'Select rp.PLAYER, (s.2018season/rp.PTS) as salary_Pts_Ratio from RegSeasonPlayer rp join Salary s on rp.PLAYER = s.Player where rp.PTS > 20 and rp.year = 2017 ORDER BY salary_Pts_Ratio LIMIT 10'
cursor.execute(sql2)
cursor.fetchall()
#There is no surprise Mitchell has the best salary-pts ratio since he's still in rookie contract,but among those have big contract, who has the worst?
# it can also be described as, whose contract is worth for the team.

(('Donovan Mitchell', 151779.51219512196),
 ('Karl-Anthony Towns', 368048.6047323381),
 ('Kemba Walker', 542986.415966838),
 ('Khris Middleton', 646766.156879517),
 ('Kyrie Irving', 823737.2669766734),
 ('Jimmy Butler', 893766.9512662131),
 ('Giannis Antetokounmpo', 898041.053627341),
 ('Anthony Davis', 905133.9023031608),
 ('Victor Oladipo', 909090.8940783266),
 ('LaMarcus Aldridge', 967403.2307777037))

In [19]:
# Query 3
# under contract, whose contract is least worth? Here we query by points only, but we can also query AST and REB either.
# The condition is that he must have a high contract. Those with vet contract will make this query a bit strange.
sql3 = 'Select rp.PLAYER, format((s.2018season/rp.PTS),0) as salary_Pts_Ratio from RegSeasonPlayer rp join Salary s on rp.PLAYER = s.Player where s.2018season>20000000 and rp.year = 2017 ORDER BY salary_Pts_Ratio desc LIMIT 10'
cursor.execute(sql3)
cursor.fetchall()


# surprise? But remember we only take points into consideration. And Paul contribute assist most!
# And you will find why Rockets wants to trade Anderson -- low contribution! But their addition Anthony is not much better... 

(('LaMarcus Aldridge', '967,403'),
 ('Victor Oladipo', '909,091'),
 ('Anthony Davis', '905,134'),
 ('Giannis Antetokounmpo', '898,041'),
 ('Kyrie Irving', '823,737'),
 ('Al Horford', '2,242,536'),
 ('Ryan Anderson', '2,195,865'),
 ('Nicolas Batum', '2,068,965'),
 ('Chris Paul', '1,916,890'),
 ('Kyle Lowry', '1,913,580'))

## Whether relation algebra works in this database? In the following queries, the first one is just cross product and in the second one ,we select first. But the speed is almost the same. 

In [6]:
import timeit
start_time = timeit.default_timer()
for i in range(1000):
    sql3 = 'Select rp.PLAYER, (s.2018season/rp.PTS) as salary_Pts_Ratio from RegSeasonPlayer rp,Salary s where rp.PLAYER = s.Player and s.2018season>20000000 and rp.year = 2017 ORDER BY salary_Pts_Ratio desc LIMIT 10'
    cursor.execute(sql3)

end_time = timeit.default_timer()
elapsed = timeit.default_timer() - start_time
elapsed



14.146420588000183

In [4]:
import timeit
start_time = timeit.default_timer()
for i in range(1000):
    sql3 = 'Select rp.PLAYER, (s.2018season/rp.PTS) as salary_Pts_Ratio from (select PLAYER,PTS FROM RegSeasonPlayer where year = 2017) rp join (select 2018season,Player from Salary where 2018season>20000000) s on rp.PLAYER = s.Player ORDER BY salary_Pts_Ratio desc LIMIT 10'
    cursor.execute(sql3)

end_time = timeit.default_timer()
elapsed = timeit.default_timer() - start_time
elapsed

14.321036563000234

In [19]:
# Query 4
#Top Players in 2019 market?
sql4 = 'select rp.PLAYER,rp.PTS,rp.year from RegSeasonPlayer rp join Salary s on rp.PLAYER = s.Player where rp.year = 2017 and s.2019season = 0 ORDER BY rp.PTS desc LIMIT 10'
cursor.execute(sql4)
cursor.fetchall()

(('Kemba Walker', 22.1, 2017),
 ('Klay Thompson', 20.0, 2017),
 ('Tobias Harris', 18.6, 2017),
 ('Eric Bledsoe', 17.7, 2017),
 ('JJ Redick', 17.1, 2017),
 ('Carmelo Anthony', 16.2, 2017),
 ('Austin Rivers', 15.1, 2017),
 ('Rodney Hood', 14.7, 2017),
 ('Zach Randolph', 14.5, 2017),
 ('Bojan Bogdanovic', 14.3, 2017))

# in the following cells, we will query whether player with highest score will help his team win championship. To simplisify it, we create a temporary table in mysql 

In [20]:
sql_temp_table1 = 'CREATE TEMPORARY TABLE IF NOT EXISTS temp_reg select rp.year, max(rp.PTS) as top_points from RegSeasonPlayer rp GROUP BY rp.year'
sql_temp_table2 = 'CREATE TEMPORARY TABLE IF NOT EXISTS temp_post select pp.year, max(pp.PTS) as top_points from PostSeasonPlayer pp GROUP BY pp.year'
#in the two following tables, we create the highest score in one season. And we will use that temporary table to do further queries.
cursor.execute(sql_temp_table1)
cursor.execute(sql_temp_table2)

18

In [21]:
cursor.execute('select * from temp_post')
cursor.fetchall()

((2005, 34.0),
 (2001, 30.8),
 (2002, 32.1),
 (2000, 33.8),
 (2004, 31.2),
 (2003, 26.6),
 (2006, 32.8),
 (2007, 30.1),
 (2009, 33.2),
 (2008, 35.3),
 (2010, 28.6),
 (2011, 30.3),
 (2012, 30.8),
 (2013, 29.6),
 (2014, 31.5),
 (2015, 28.4),
 (2016, 37.4),
 (2017, 34.0))

In [22]:
#uery 5
sql5 = 'select rp.year,rp.PLAYER,pt.TEAM_NAME \
from (RegSeasonPlayer rp join temp_reg on rp.year = temp_reg.year and rp.PTS = temp_reg.top_points) \
join PostSeasonTeam pt on pt.TEAM_NAME = rp.TEAM where pt.W = 16 and rp.year = pt.year'
cursor.execute(sql5)
cursor.fetchall()
#surprisingly, those with highest point in regseason don't win the championship that year!

()

In [23]:
#Query 6
sql6 = 'select pp.year,pp.PLAYER,pt.TEAM_NAME \
from (PostSeasonPlayer pp join temp_post on pp.year = temp_post.year and pp.PTS = temp_post.top_points) \
join PostSeasonTeam pt on pt.TEAM_NAME = pp.TEAM where pt.W = 16 and pt.year = pp.year'
cursor.execute(sql6)
cursor.fetchall()

#only James win both title and highest points in postseason

((2011, 'LeBron James', 'Miami Heat'),)

In [24]:
#Query 7
# best shooting group in Reg for each season

sql7 = 'SELECT p1.year,p1.PLAYER,p1.FG3_PCT,p1.TEAM \
FROM RegSeasonPlayer p1 join \
(SELECT t1.year,t1.TEAM_NAME FROM RegSeasonTeam t1 \
WHERE (t1.FG3_PCT,t1.year) in (SELECT MAX(t2.FG3_PCT),t2.year FROM RegSeasonTeam t2 GROUP BY t2.year)) t \
on t.year = p1.year and p1.TEAM = t.TEAM_NAME \
WHERE 3>\
(SELECT COUNT(DISTINCT p2.PLAYER) \
FROM RegSeasonPlayer p2 \
WHERE p2.FG3_PCT > p1.FG3_PCT AND p1.TEAM = p2.TEAM AND p1.year = p2.year)\
ORDER BY p1.year DESC,p1.FG3_PCT DESC \
'
cursor.execute(sql7)
cursor.fetchall()

((2017, 'Klay Thompson', 0.44, 'Golden State Warriors'),
 (2017, 'Kevin Durant', 0.419, 'Golden State Warriors'),
 (2017, 'Nick Young', 0.377, 'Golden State Warriors'),
 (2016, 'Pau Gasol', 0.538, 'San Antonio Spurs'),
 (2016, 'Patty Mills', 0.414, 'San Antonio Spurs'),
 (2016, 'LaMarcus Aldridge', 0.411, 'San Antonio Spurs'),
 (2015, 'Andrew Bogut', 1.0, 'Golden State Warriors'),
 (2015, 'Stephen Curry', 0.454, 'Golden State Warriors'),
 (2015, 'Klay Thompson', 0.425, 'Golden State Warriors'),
 (2014, 'Stephen Curry', 0.443, 'Golden State Warriors'),
 (2014, 'Klay Thompson', 0.439, 'Golden State Warriors'),
 (2014, 'Harrison Barnes', 0.405, 'Golden State Warriors'),
 (2013, 'Marco Belinelli', 0.43, 'San Antonio Spurs'),
 (2013, 'Matt Bonner', 0.429, 'San Antonio Spurs'),
 (2013, 'Patty Mills', 0.425, 'San Antonio Spurs'),
 (2012, 'Stephen Curry', 0.453, 'Golden State Warriors'),
 (2012, 'Jarrett Jack', 0.404, 'Golden State Warriors'),
 (2012, 'Klay Thompson', 0.401, 'Golden State Warr

In [25]:
# Query 8
# best shooting group in Post
cursor.execute('SELECT p1.year,p1.PLAYER,p1.FG3_PCT,p1.TEAM \
FROM PostSeasonPlayer p1 join \
(SELECT t1.year,t1.TEAM_NAME FROM PostSeasonTeam t1 \
WHERE (t1.FG3_PCT,t1.year) in (SELECT MAX(t2.FG3_PCT),t2.year FROM PostSeasonTeam t2 GROUP BY t2.year)) t \
on t.year = p1.year and p1.TEAM = t.TEAM_NAME \
WHERE 3>\
(SELECT COUNT(DISTINCT p2.PLAYER) \
FROM PostSeasonPlayer p2 \
WHERE p2.FG3_PCT > p1.FG3_PCT AND p1.TEAM = p2.TEAM AND p1.year = p2.year)\
ORDER BY p1.year DESC,p1.FG3_PCT DESC \
')
cursor.fetchall()

((2017, 'Derrick Rose', 0.7, 'Minnesota Timberwolves'),
 (2017, 'Nemanja Bjelica', 0.571, 'Minnesota Timberwolves'),
 (2017, 'Jimmy Butler', 0.471, 'Minnesota Timberwolves'),
 (2016, 'JR Smith', 0.5, 'Cleveland Cavaliers'),
 (2016, 'Kevin Love', 0.45, 'Cleveland Cavaliers'),
 (2016, 'LeBron James', 0.411, 'Cleveland Cavaliers'),
 (2015, 'LaMarcus Aldridge', 1.0, 'San Antonio Spurs'),
 (2015, 'David West', 0.5, 'San Antonio Spurs'),
 (2015, 'Danny Green', 0.5, 'San Antonio Spurs'),
 (2014, 'Paul Pierce', 0.524, 'Washington Wizards'),
 (2014, 'Drew Gooden', 0.462, 'Washington Wizards'),
 (2014, 'Ramon Sessions', 0.4, 'Washington Wizards'),
 (2013, 'Danny Green', 0.475, 'San Antonio Spurs'),
 (2013, 'Marco Belinelli', 0.421, 'San Antonio Spurs'),
 (2013, 'Kawhi Leonard', 0.419, 'San Antonio Spurs'),
 (2012, 'Klay Thompson', 0.424, 'Golden State Warriors'),
 (2012, 'Stephen Curry', 0.396, 'Golden State Warriors'),
 (2012, 'Draymond Green', 0.391, 'Golden State Warriors'),
 (2011, 'Stephen 

In [16]:
#Query 9
# who's the team hero?That is to say, the ratio of his points/team-points is very high. He contributes most to his team.

sql8 = 'select p.PLAYER,t.TEAM_NAME,format(sum(p.GP*p.PTS),2) AS PLAYER_PT,format(sum((t.W+t.L)*t.PTS),2) AS TEAM_PT,format((sum(p.GP*p.PTS)/sum((t.W+t.L)*t.PTS)),2) AS CONT \
FROM RegSeasonPlayer p join RegSeasonTeam t on p.year = t.year and p.TEAM = t.TEAM_NAME \
GROUP BY p.PLAYER, t.TEAM_NAME \
ORDER BY CONT DESC \
LIMIT 10'
cursor.execute(sql8)
cursor.fetchall()

(('Allen Iverson', 'Philadelphia 76ers', '11,033.80', '39,442.00', '0.28'),
 ('Kevin Durant', 'Oklahoma City Thunder', '15,247.50', '58,087.40', '0.26'),
 ('Tracy McGrady', 'Orlando Magic', '8,292.70', '32,021.00', '0.26'),
 ('Jerry Stackhouse', 'Detroit Pistons', '4,010.40', '15,571.80', '0.26'),
 ('Gilbert Arenas', 'Washington Wizards', '6,485.60', '25,133.00', '0.26'),
 ('Kobe Bryant', 'Los Angeles Lakers', '28,533.80', '114,670.00', '0.25'),
 ('LeBron James', 'Miami Heat', '7,913.00', '31,691.40', '0.25'),
 ('LeBron James', 'Cleveland Cavaliers', '23,121.80', '91,118.40', '0.25'),
 ('James Harden', 'Houston Rockets', '13,017.70', '53,447.60', '0.24'),
 ('Vince Carter', 'New Jersey Nets', '9,147.50', '39,089.40', '0.23'))

In [17]:
#Query 10
#under rated 3pt shooter

sql = 'SELECT rp.PLAYER , rp.FG3_PCT,r.3Pt,format((rp.FG3_PCT/r.3Pt)*100,2) as ratio \
FROM RegSeasonPlayer rp join Ratings r on rp.PLAYER = r.Player \
WHERE rp.year = 2017 and r.3Pt > 75 \
ORDER BY ratio DESC LIMIT 20'
cursor.execute(sql)
cursor.fetchall()

(('Darren Collison', 0.468, 76.0, '0.62'),
 ('Otto Porter Jr.', 0.441, 80.0, '0.55'),
 ('Marvin Williams', 0.413, 76.0, '0.54'),
 ('Joe Ingles', 0.44, 82.0, '0.54'),
 ('Jonas Jerebko', 0.414, 77.0, '0.54'),
 ('Luke Kennard', 0.415, 77.0, '0.54'),
 ('Buddy Hield', 0.431, 81.0, '0.53'),
 ('George Hill', 0.415, 78.0, '0.53'),
 ('Kevin Love', 0.415, 80.0, '0.52'),
 ('Dirk Nowitzki', 0.409, 79.0, '0.52'),
 ('Courtney Lee', 0.406, 78.0, '0.52'),
 ('Kyrie Irving', 0.408, 79.0, '0.52'),
 ('Doug McDermott', 0.426, 82.0, '0.52'),
 ('Kevin Durant', 0.419, 81.0, '0.52'),
 ('Garrett Temple', 0.392, 76.0, '0.52'),
 ('Bojan Bogdanovic', 0.402, 78.0, '0.52'),
 ("E'Twaun Moore", 0.425, 84.0, '0.51'),
 ('Tony Snell', 0.403, 79.0, '0.51'),
 ('Paul George', 0.401, 79.0, '0.51'),
 ('Kyle Lowry', 0.399, 78.0, '0.51'))

In [2]:
#Query 11
#common teammates of two certain player

def common_teammates(P1,P2):
    sql = "SELECT T1.Player \
    FROM (select DISTINCT P1.PLAYER \
    FROM RegSeasonPlayer P1 join RegSeasonPlayer P2 ON P1.year = P2.year and P1.TEAM =P2.TEAM \
    WHERE P2.PLAYER = " + "'" + P1 + "'" + "  AND P1.PLAYER != "+ "'" + P1 + "') T1" +" join (SELECT DISTINCT P3.PLAYER \
    FROM RegSeasonPlayer P3 join RegSeasonPlayer P4 ON P3.year = P4.year and P3.TEAM = P4.TEAM \
     WHERE P4.PLAYER = " + "'" + P2 +"'"+" AND P3.PLAYER != "+ "'"+P2+ "') T2 on T1.PLAYER = T2.PLAYER"
#     print(sql)
    cursor.execute(sql)
    return cursor.fetchall()
common_teammates('Lebron James','Chris Paul')

(('Jeff Green',),)

In [29]:
#close the cursor
cursor.close()
db.close()

In [22]:
# a final query we create for our fantasy game. We will get name from frontend, and get information from backend. And if possible, we will store
# information into users mongodb database. 
name = 'James Harden'
sql = "SELECT s.2018season, r.Player,r.Speed,r.Midrange,r.Finishing,r.3Pt,r.Dribble,r.PostScoring,r.Rebound,r.Defense \
FROM Ratings r join Salary s on r.Player = s.Player where s.Player = " +"'"+ name +"'"
cursor.execute(sql)
cursor.fetchall()

((30570000, 'James Harden', 82.0, 85.0, 90.0, 77.0, 85.0, 66.0, 76.0, 72.0),)